# Notes on Struct and StructArray 

Based on [this post](https://discourse.julialang.org/t/understanding-and-avoiding-allocations-with-structarrays/46880/3).

In [2]:
using LinearAlgebra 
using SparseArrays 
using StructArrays
using StaticArrays

using IterativeSolvers
using Preconditioners

using BenchmarkTools
using Profile
using ProfileView

using Plots 

## Section 1: Parametrized Struct 

Assign a 3-vector to a point. 

In [19]:
struct Foo{T}
    idx::Int 
    vec::SVector{3,T}
end

fInt = Foo{Int}(2, @SVector [-1,0,1])

display(fInt)
display(getfield(fInt, 1))
display(getfield(fInt, 2))

fFloat = Foo{Float64}(2, @SVector [-1.,0.,1.])

display(fFloat)
display(getfield(fFloat, 1))
display(getfield(fFloat, 2))

Foo{Int64}(2, [-1, 0, 1])

2

3-element SVector{3, Int64} with indices SOneTo(3):
 -1
  0
  1

Foo{Float64}(2, [-1.0, 0.0, 1.0])

2

3-element SVector{3, Float64} with indices SOneTo(3):
 -1.0
  0.0
  1.0

In [21]:
?getproperty

search: getproperty



```
getproperty(value, name::Symbol)
getproperty(value, name::Symbol, order::Symbol)
```

The syntax `a.b` calls `getproperty(a, :b)`. The syntax `@atomic order a.b` calls `getproperty(a, :b, :order)` and the syntax `@atomic a.b` calls `getproperty(a, :b, :sequentially_consistent)`.

# Examples

```jldoctest
julia> struct MyType
           x
       end

julia> function Base.getproperty(obj::MyType, sym::Symbol)
           if sym === :special
               return obj.x + 1
           else # fallback to getfield
               return getfield(obj, sym)
           end
       end

julia> obj = MyType(1);

julia> obj.special
2

julia> obj.x
1
```

See also [`getfield`](@ref Core.getfield), [`propertynames`](@ref Base.propertynames) and [`setproperty!`](@ref Base.setproperty!).


In [20]:
getproperty(getfield(fFloat, 1), :vec)
# getproperty(getfield(fFloat, 1), :vec)[getfield(f, 2)] = val

LoadError: type Int64 has no field vec

## Section 2: StructArray 

In [22]:
struct Foo{T}
    idx::Int 
    vec::SVector{3,T}
end

#..structarray holds points and 3-vectors 
sarr = StructArray(Foo{Float64}(i, zeros(3)) for i in 1:5)

#..constant 3-vector
val = @SVector [1.,2.,3.]

#..assign given 3-vector to each point of array:: version-1:: allocing in older julia 
function testalloc(sarr, val)
    for f in LazyRows(sarr)
        f.vec = val
    end
end

# This function does not allocate, I just replaced the f.vec call by the inlined equivalent.
function testnoalloc(sarr, val)
    for f in LazyRows(sarr)
        display(getproperty(getfield(f, 1), :vec))
        getproperty(getfield(f, 1), :vec)[getfield(f, 2)] = val
    end
end

testnoalloc (generic function with 1 method)

In [24]:
testnoalloc(sarr, val)
sarr

5-element Vector{SVector{3, Float64}}:
 [1.0, 2.0, 3.0]
 [1.0, 2.0, 3.0]
 [1.0, 2.0, 3.0]
 [1.0, 2.0, 3.0]
 [1.0, 2.0, 3.0]

5-element Vector{SVector{3, Float64}}:
 [1.0, 2.0, 3.0]
 [1.0, 2.0, 3.0]
 [1.0, 2.0, 3.0]
 [1.0, 2.0, 3.0]
 [1.0, 2.0, 3.0]

5-element Vector{SVector{3, Float64}}:
 [1.0, 2.0, 3.0]
 [1.0, 2.0, 3.0]
 [1.0, 2.0, 3.0]
 [1.0, 2.0, 3.0]
 [1.0, 2.0, 3.0]

5-element Vector{SVector{3, Float64}}:
 [1.0, 2.0, 3.0]
 [1.0, 2.0, 3.0]
 [1.0, 2.0, 3.0]
 [1.0, 2.0, 3.0]
 [1.0, 2.0, 3.0]

5-element Vector{SVector{3, Float64}}:
 [1.0, 2.0, 3.0]
 [1.0, 2.0, 3.0]
 [1.0, 2.0, 3.0]
 [1.0, 2.0, 3.0]
 [1.0, 2.0, 3.0]

5-element StructArray(::Vector{Int64}, ::Vector{SVector{3, Float64}}) with eltype Foo{Float64}:
 Foo{Float64}(1, [1.0, 2.0, 3.0])
 Foo{Float64}(2, [1.0, 2.0, 3.0])
 Foo{Float64}(3, [1.0, 2.0, 3.0])
 Foo{Float64}(4, [1.0, 2.0, 3.0])
 Foo{Float64}(5, [1.0, 2.0, 3.0])

In [5]:
@btime testalloc($sarr, $val)
@btime testnoalloc($sarr, $val)

  513.453 ns (0 allocations: 0 bytes)
  513.672 ns (0 allocations: 0 bytes)
